In [11]:
import os

os.environ["DNNBRAIN_DATA"] = "data"

!dnn_act -net AlexNet -layer conv1_relu conv2_relu conv3_relu conv4_relu conv5_relu -stim data/all_5000scenes.stim.csv -out AlexNet_relu.act.h5

/Users/weichen/PycharmProjects/RSA_CNN/venv/lib/python3.9/site-packages/dnnbrain-1.0a0-py3.9.egg/dnnbrain/dnn/models.py:902: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  se

In [12]:
from scipy.stats import zscore
from dnnbrain.dnn.core import Activation

activ_file = 'AlexNet_relu.act.h5'
out_file = 'AlexNet_relu_zscore.act.h5'

activ = Activation()
activ.load(activ_file)
for layer in activ.layers:
    activ_arr = activ.get(layer)
    shape = activ_arr.shape
    activ_arr = activ_arr.reshape((shape[0], -1))
    activ_arr = zscore(activ_arr, axis=1)
    activ.set(layer, activ_arr.reshape(shape))
activ.save(out_file)

error: Error -5 while decompressing data: incomplete or truncated stream

In [ ]:
!dnn_fe -act AlexNet_relu_zscore.act.h5 -meth pca 100 -out AlexNet_relu_zscore_PCA-100.act.h5
!dnn_rsa -act AlexNet_relu_zscore_PCA-100.act.h5 -metric correlation -out AlexNet_relu_zscore_PCA-100.rdm.h5
!brain_rsa -nif beta_rh_all_run.nii.gz -bmask VTC_mask_rh.nii.gz -metric correlation -out beta_rh_VTC.rdm.h5

In [ ]:
import numpy as np
from dnnbrain.dnn.core import RDM, Stimulus
from dnnbrain.utils.plot import imgarray_show

# load RDMs
brdm = RDM()
brdm.load('beta_rh_VTC.rdm.h5')
drdm = RDM()
drdm.load('AlexNet_relu_zscore_PCA-100.rdm.h5')

# get rearrange indices
stim = Stimulus()
stim.load('all_5000scenes.stim.csv')
labels = stim.get('label')
labels_uniq = np.unique(labels)
indices = []
for lbl in labels_uniq:
    indices.extend(np.where(labels == lbl)[0])

# get brain RDM
brdm_arr = brdm.get('1', False)
brdm_arr = brdm_arr + brdm_arr.T
rdm_arrs = [brdm_arr[indices][:, indices]]
img_names = ['VTC']

# get DNN RDMs
layers = [f'conv{i}_relu' for i in range(1, 6)]
img_names.extend(layers)
for layer in layers:
    drdm_arr = drdm.get(layer, False)[0]
    drdm_arr = drdm_arr + drdm_arr.T
    rdm_arrs.append(drdm_arr[indices][:, indices])

# plot
imgarray_show(rdm_arrs, 2, 3, cmap='hot', cbar=True,
              frame_on=False, img_names=img_names)

In [ ]:
from scipy.stats import pearsonr
from dnnbrain.dnn.core import RDM

# load RDMs
brdm = RDM()
brdm.load('beta_rh_VTC.rdm.h5')
drdm = RDM()
drdm.load('AlexNet_relu_zscore_PCA-100.rdm.h5')

# calculate correlation between DNN RDMs and brain RDM.
layers = [f'conv{i}_relu' for i in range(1, 6)]
brdm_arr = brdm.get('1', True)
for idx, layer in enumerate(layers):
    drdm_arr = drdm.get(layer, True)[0]
    corr = pearsonr(brdm_arr, drdm_arr)[0]
    print(f'VTC corr {layer}: {corr}')